# Parameter Estimation: FitzHugh Example

## Defining the model

We are going to investigate another classic model here, the
FitzHugh-Nagumo, or simply FitzHugh here. The model has already been
defined in {mod}`common_models` so we can load it rather than define by scratch.

#TODO ref for FH-N

In [ ]:
from pygom import SquareLoss, common_models

import numpy

import scipy.integrate, scipy.optimize

import math,time,copy

import matplotlib.pyplot as plt

x0 = \[-1.0, 1.0\]

t0 = 0

# params
paramEval = [('a',0.2), ('b',0.2), ('c',3.0)]

ode = common_models.FitzHugh(paramEval)

ode.initial_values = (x0, t0)



Define a set of time points and we can see how the two states $V$ and $R$
are suppose to behave.



In [ ]:
t = numpy.linspace(1, 20, 30).astype('float64')

solution = ode.integrate(t)

In [ ]:
ode.plot()


## Estimate the parameters

Obtaining the correct parameters for the FitzHugh model is well known to
be difficult, because of its multimodal surface. Although this
has been shown many times in the literature, so we will omit the
details. For further details see {citets}'FitzHugh`.

#TODO ref?

We will give the fitting a go with an initial guess which will enable us to recover the original parameters. First, we try the fitting process with only one target state.



In [ ]:
theta = [0.5, 0.5, 0.5]

objFH = SquareLoss(theta, ode, x0, t0, t, solution[1::,1], 'R')

boxBounds = [(0.0,5.0), (0.0,5.0), (0.0,5.0)]

res = scipy.optimize.minimize(fun=objFH.cost, jac=objFH.sensitivity, 
                              x0=theta, bounds=boxBounds,
                              method='L-BFGS-B')

print(res)

Then we try the same again but with both states as our target. 

objFH = SquareLoss(theta, ode, x0, t0, t, solution[1::,:], ['V','R'])

res = scipy.optimize.minimize(fun=objFH.cost, jac=objFH.sensitivity, x0=theta,
                              bounds=boxBounds, method='L-BFGS-B')
print(res)



Note how the estimates are the same, unlike other models.

#TODO why is this?



## Estimating initial value

We can further assume that we have no idea about the initial values for
$V$ and $R$ as well. We also provide a guesstimate to set off the
optimization. The input vector $\theta$ must have the parameters first,
then the initial values, along with the corresponding bounds.

First, only a single target state, i.e. we only have observations for
one of states which is $R$ in this case

In [ ]:
objFH = SquareLoss(theta, ode, x0, t0, t, solution[1::,1], 'R')

boxBounds = [(0.0,5.0), 
             (0.0,5.0),
             (0.0,5.0),
             (None,None),
             (None,None)]

res = scipy.optimize.minimize(fun=objFH.costIV,
                              jac=objFH.sensitivityIV,
                              x0=theta + [-0.5,0.5],
                              bounds=boxBounds, 
                              method='L-BFGS-B')

print(res)


Then we can find both states as target at the same time.


In [ ]:
objFH = SquareLoss(theta, ode, x0, t0, t, solution[1::,:], ['V','R'])

res = scipy.optimize.minimize(fun=objFH.costIV, 
                              jac=objFH.sensitivityIV, 
                              x0=theta + [-0.5, 0.5],
                              bounds=boxBounds, 
                              method='L-BFGS-B')

print(res)



See the difference between the two estimates with the latter method; both states
were used, yielding superior estimates. Note that only the forward
sensitivity method is implemented when estimating the initial value, and
it is assumed that the starting condition for all the states are
unknown.

The choice of algorithm here is the **L-BFGS-B** which is a better
choice because the parameter space of the FitzHugh is rough (i.e. large
second derivative) as well as being multimodal. This means that the
Hessian is not guaranteed to be positive definite and approximation
using $J^{\top}J$ is poor, with $J$ being the Jacobian of the objective
function.